# Explore here

It's recommended to use this notebook for exploration purposes.

In [99]:
pip install pandas requests lxml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [100]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [101]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns



In [102]:
# import io

url = "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify"

# Send HTTP GET request to the URL
response = requests.get(url)

# Check if response is succesful
if response.status_code == 200:

    html = response.text
    print(f"Status: {response.status_code}")

else:
    print(f"Failed. Code: {response.status_code}")

Status: 200


In [103]:
# Parse HTML content
soup = BeautifulSoup(html, "html.parser")

# Find all tables
tables = soup.find_all("table")

# print(tables)

In [104]:
import pandas as pd

# Use pandas to read all tables on the page
tables = pd.read_html(response.text)

# The first table on the page is usually the one we want
df = tables[0]

# Preview the first few rows
df.head()

/tmp/ipykernel_642/2187473486.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,"""Blinding Lights""",The Weeknd,4.936,29 November 2019,[2]
1,2,"""Shape of You""",Ed Sheeran,4.464,6 January 2017,[3]
2,3,"""Starboy""",The Weeknd and Daft Punk,3.998,21 September 2016,[4]
3,4,"""Someone You Loved""",Lewis Capaldi,3.981,8 November 2018,[5]
4,5,"""As It Was""",Harry Styles,3.972,1 April 2022,[6]


In [105]:
# Clean the rows by removing row Ref.

df = df.drop(columns=['Ref.'])

df.head()

,Rank,Song,Artist(s),Streams (billions),Release date
0,1,"""Blinding Lights""",The Weeknd,4.936,29 November 2019
1,2,"""Shape of You""",Ed Sheeran,4.464,6 January 2017
2,3,"""Starboy""",The Weeknd and Daft Punk,3.998,21 September 2016
3,4,"""Someone You Loved""",Lewis Capaldi,3.981,8 November 2018
4,5,"""As It Was""",Harry Styles,3.972,1 April 2022


In [106]:
# Create database connection
conn = sqlite3.connect("spotify_streams.db")

# Drop existing top_songs table (IDK where its exist?)
conn.execute("DROP TABLE IF EXISTS top_songs")

# Create the table 
conn.execute("""
    CREATE TABLE top_songs (
        Rank INTEGER,
        Song TEXT,
        Artist TEXT,
        Streams INTEGER,
        Release_date TEXT
    )
""")

# Removes unecesary rows in column Rank
df = df[df["Rank"].apply(lambda x: str(x).isdigit())]

# Rename columns to match table names
df.columns = ["Rank", "Song", "Artist(s)", "Streams (billions)", "Release date"]

# Insert rows
for _, row in df.iterrows():
    conn.execute('''
        INSERT INTO top_songs (Rank, Song, Artist, Streams, Release_date)
        VALUES (?, ?, ?, ?, ?)
    ''', (int(row["Rank"]), row["Song"], row["Artist(s)"], float(row["Streams (billions)"]), row["Release date"]))

# Store in SQLite database
conn.commit()
conn.close()